In [341]:
using RigidBodyDynamics
using GeometryTypes
using AffineTransforms
using LCMGL
using Interact
using ForwardDiff
using DataFrames
using SpatialFields
import RigidBodyTreeInspector
import DrakeVisualizer: Visualizer, draw
import DataStructures: OrderedDict

In [342]:
include("arms.jl")
include("depth_sensors.jl")

DepthSensors

In [357]:
function beanbag()
    limbs = OrderedDict{RigidBody, Arms.Limb}()
    
    mechanism = Mechanism{Float64}("world")
    parent = root_body(mechanism)
    
    joint = Joint("joint1", QuaternionFloating())
    joint_to_parent = Transform3D(Float64, joint.frameBefore, parent.frame)
    body = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")))
    body_to_joint = Transform3D(Float64, body.frame, joint.frameAfter)
    attach!(mechanism, parent, joint, joint_to_parent, body, body_to_joint)
    
    surface_points = Vector{Point3D}()
    skeleton_points = [Point3D(body.frame, Vec(0.,0,0))]
    for axis = 1:3
        for s = [-1; 1]
            x = [0.; 0; 0]
            x[axis] = s
            push!(surface_points, Point3D(body.frame, Vec(x)))
        end
    end
    limbs[body] = Arms.Limb(surface_points, skeleton_points)
    
    Arms.Model(mechanism, limbs)
end

beanbag (generic function with 1 method)

In [358]:
model = beanbag()

Arms.Model(Vertex: world (root)
  Vertex: body1, Edge: joint1,DataStructures.OrderedDict{RigidBodyDynamics.RigidBody{T<:Real},Arms.Limb}(RigidBody: "body1"=>Arms.Limb(RigidBodyDynamics.Point3D[Point3D in "body1": Vec(-1.0,0.0,0.0),Point3D in "body1": Vec(1.0,0.0,0.0),Point3D in "body1": Vec(0.0,-1.0,0.0),Point3D in "body1": Vec(0.0,1.0,0.0),Point3D in "body1": Vec(0.0,0.0,-1.0),Point3D in "body1": Vec(0.0,0.0,1.0)],RigidBodyDynamics.Point3D[Point3D in "body1": Vec(0.0,0.0,0.0)])))

In [359]:
state = Arms.ModelState(model)
draw(model, state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot([DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{Float64,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5966xFixedSizeArrays.Point{3,Float64},     faces: 11928xGeometryTypes.Face{3,Int64,0}, )
,AffineTransforms.AffineTransform{Float64,3}:
matrix: 3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
translation: [0.0,0.0,0.0]
,RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7effc0a367e0>))

In [360]:
state.limb_deformations[1][1].frame

CartesianFrame3D: "body1"

In [361]:
joint_angles, deformations = Arms.zero_configuration(model)
state = Arms.ModelState(model, joint_angles, deformations)
@manipulate for x = linspace(-1, 1, 51)
    state.limb_deformations[1][1] = FreeVector3D(model.mechanism.toposortedTree[2].vertexData.frame, Vec(x, 0, 0))
    draw(model, state)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),true)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot([DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{Float64,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5966xFixedSizeArrays.Point{3,Float64},     faces: 11928xGeometryTypes.Face{3,Int64,0}, )
,AffineTransforms.AffineTransform{Float64,3}:
matrix: 3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
translation: [0.0,0.0,0.0]
,RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7effc0a36ed0>))

In [390]:
# Construct the sensor and draw its view rays for debugging
sensor = DepthSensors.Kinect(80,80);
camera_origin = tformtranslate([0;0;8]) * tformrotate([pi;0;0])
LCMGLClient("sensor_rays") do lcmgl
    LCMGL.color(lcmgl, 0, 1, 0)
    begin_mode(lcmgl, LCMGL.LINES)
    for ray in sensor.rays
        vertex(lcmgl, camera_origin.offset...)
        vertex(lcmgl, (camera_origin.offset + camera_origin.scalefwd * convert(Vector, ray))...)
    end
    end_mode(lcmgl)
    switch_buffer(lcmgl)
end

In [391]:
function raycast(model::Arms.Model, state::Arms.ModelState)
    draw(model, state, true)
    skin = Arms.skin(model, state)
    
    points = DepthSensors.raycast_points(skin, sensor, camera_origin)
    LCMGLClient("raycast") do lcmgl
        LCMGL.color(lcmgl, 0, 1, 0)
        point_size(lcmgl, 5)
        begin_mode(lcmgl, LCMGL.POINTS)
        for point in points
            vertex(lcmgl, convert(Vector, point)...)
        end
        end_mode(lcmgl)
        switch_buffer(lcmgl)
    end
    points
end

raycast (generic function with 17 methods)

In [392]:
joint_angles, deformations = Arms.zero_configuration(model)
joint_angles[5:7] = 2*rand(3)
for (i, limb_deformations) in enumerate(deformations)
    for (j, deformation) in enumerate(limb_deformations)
        deformations[i][j] = rand(Vec{3, Float64}) - 0.5
    end
end
state = Arms.ModelState(model, joint_angles, deformations)
skin = Arms.skin(model, state)

SpatialFields.InterpolatingSurface{3,Float64,SpatialFields.XSquaredLogX}([0.9427934116142082,0.3284196857048755,-0.06615259279209033,-0.11010562795842964,0.5466924016982715,0.08146585244565181,-1.723113130712487],7-elementArray{FixedSizeArrays.Point{3,Float64},1}:
 Point(0.7554867465565194,0.6094216130960253,1.8864941120228953)
 Point(2.2237201112892793,0.7179704671413012,1.9239193566112849)
 Point(1.1310120753903616,-0.7842325461255972,1.5190847551370816)
 Point(0.7951111597648939,1.6575509303329863,1.918518329905621)
 Point(1.4848242945363304,0.019659954969980298,1.0915858627808426)
 Point(1.2121473877175304,0.09931053118296473,2.8585215170654648)
 Point(1.2713654915136225,0.4054092559189906,1.6994413695236594)
,MultiPoly.MPoly{Float64}(-0.46031098629401856 - 0.013326493877439068x - 0.08220929499530905y - 0.10485649752129579z),SpatialFields.XSquaredLogX())

In [393]:
function flatten(state::Arms.ModelState)
    data = vcat(values(state.mechanism_state.q)...)
    for limb_deformation in state.limb_deformations
        for d in limb_deformation
            for x in d.v
                push!(data, x)
            end
        end
    end
    data
end

function unflatten(model::Arms.Model, state::Arms.ModelState, data)
    data_index = 1
    for joint in keys(state.mechanism_state.q)
        nq = RigidBodyDynamics.num_positions(joint)
        state.mechanism_state.q[joint][:] = data[data_index:(data_index+nq-1)]
        data_index += nq
    end
    for (i, (body, limb)) in enumerate(model.limbs)
        limb_deformation = state.limb_deformations[i]
        for (j, deformation) in enumerate(limb_deformation)
            state.limb_deformations[i][j] = RigidBodyDynamics.FreeVector3D(body.frame, Vec(data[data_index:data_index+2]))
            data_index += 3
        end
    end
    @assert data_index == length(data) + 1
    RigidBodyDynamics.setdirty!(state.mechanism_state)
    state
end

unflatten (generic function with 17 methods)

In [428]:
joint_angles, deformations = Arms.zero_configuration(model)
joint_angles[5:7] = rand(3)
for (i, limb_deformations) in enumerate(deformations)
    for (j, deformation) in enumerate(limb_deformations)
        deformations[i][j] = 0.5*(rand(Vec{3, Float64}) - 0.5)
    end
end

true_state = Arms.ModelState(model, joint_angles, deformations)
x_true = flatten(true_state)



sensed_points = raycast(model, true_state)

x_ad = []
state_ad = []

function cost{T}(x::Vector{T})
    push!(x_ad, x)
    state = Arms.ModelState(model, T, T)
    unflatten(model, state, x)
    push!(state_ad, state)
    draw(model, state, false)
    skin = Arms.skin(model, state)
    raycast_cost = sum(point -> evaluate(skin, point)^2, sensed_points)
    deformation_cost = zero(T)
    for ld in state.limb_deformations
        for d in ld
            deformation_cost += sum(d.v .^2)
        end
    end
    return raycast_cost + 10*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
g_cost = ForwardDiff.gradient(cost)

trials = []

function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = flatten(Arms.ModelState(model))

        # Now we use gradient descent to recover the joint angles
        for i = 1:500
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            gi = g_cost(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end

            # Descend the gradient
            x_estimated -= 0.002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [429]:
test_gradient_descent()
state = unflatten(model, Arms.ModelState(model), trials[1][:x][end])
draw(model, state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot([DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{Float64,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5540xFixedSizeArrays.Point{3,Float64},     faces: 11076xGeometryTypes.Face{3,Int64,0}, )
,AffineTransforms.AffineTransform{Float64,3}:
matrix: 3x3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
translation: [0.0,0.0,0.0]
,RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin")]),1,PyLCM.LCM(PyObject <LCM object at 0x7effc0a557b0>))